 # action별 sentiment, importance 산출

마이크로세그먼트에서 LDA로 도출된 각 action 별로 sentiment와 importance를 구함 (sentiment구하는데 오래걸림)

In [2]:
import pandas as pd

from tqdm import tqdm #작업진행률 표시
from konlpy.tag import Okt #okt 형태소 분석기
okt = Okt()

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer #vectorizer 
from sklearn.decomposition import LatentDirichletAllocation #lda
import numpy as np

import json #json 파일 사용

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
def preprocess_okt_for_sentiment(text): #okt 전처리 함수 정의
    #     text = spacing(text) # 띄어쓰기 보정 
    pos_words = okt.pos(text, stem=True)
    words = [word for word, tag in pos_words if tag in ['Noun', 'Adjective', 'Verb', 'KoreanParticle', 'VerbPrefix'] ]
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','요']
    stopped_words = [w for w in words if not w in stopwords]
    x ='' 
    for t in range(len(stopped_words)): #
        x = x+' '+stopped_words[t]
    return x

### 계산값 도출할 함수 정의

- 1. token(단어)의 감성점수 도출
- 2. 1을 가지고 sentence(문장)의 감성점수 도출
- 3. 2를 가지고 document(문단)의 감성점수 도출

#### 1. token(단어)의 감성점수 도출

In [1]:
#감성점수 계산 함수, 문장 평가 함수 정의

'sentiment'

def score_word(token): #token의 감성점수를 계산 -2(부정) ~ 2점 (긍정)
    with open('SentiWord_info.json', encoding='utf-8-sig', mode='r') as f: # KNU 감성사전
        data = json.load(f)
    result = ['None','None']
    for i in range(0, len(data)):
        if data[i]['word'] == token:
            result.pop()
            result.pop()
            result.append(data[i]['word_root'])
            result.append(data[i]['polarity'])
    return result[1]

In [75]:
#위 함수 도출 예시
#token에는 okt로 토큰화된 값이, test에는 감성 값이 들어감

tokens = okt.pos('나는 하루종일 기분이 슬프고 안좋아 못해 않좋아', stem = True) 
token = []
test = []
for i in tokens:
    token.append(i[0])
    test.append(score_word(i[0]))
token, test

(['나', '는', '하루', '종일', '기분', '이', '슬프다', '안좋다', '못', '하다', '않다', '좋다'],
 ['None',
  'None',
  'None',
  'None',
  'None',
  'None',
  '-2',
  'None',
  '0',
  'None',
  'None',
  '2'])

#### 1을 가지고 sentence(문장)의 감성점수 도출

In [8]:
# 문장을 평가하는 함수 : 각 문장 평가후 문단 감성점수 도출할 것임
# sentence = 문자열 문장
def score_sentence(sentence):
    tokens = okt.pos(sentence, stem = True) #okt pos tagging 결과물 "안녕하세요" -> '안녕하다', a'djective'
    
    token = [] #okt로 분리된 토큰이 들어감
    test = [] #토큰의 감성점수가 들어감
    for i in tokens:
        token.append(i[0]) 
        test.append(score_word(i[0]))
    
    
    #비지도방식 감성분석에서 문제되는 부분 해결
    #안좋다 -> 부정이 되는데 '안 좋다'의 경우 안 -> 중립 좋다-> 긍정으로 나옴
    #따라서 위의 경우를 직접 부정으로 바꿔줌
    #안 좋다 (짧은 부정문) 좋지 않다 (긴 부정문)
    
    prenegative = ["안", "못"] #짧은 부정문 처리
    sunegative = ["않다", "모르다"] #긴 부정문 처리
    
    adjust = []
    #enumerate는 for 루프를 돌릴 때, 인덱스와 원소를 동시에 얻음
    for a, i in enumerate(test): # i에는 각 토큰이 나오고, a에는 인덱스가 나옴
        try: # 첫번째 토큰일 경우 [a-1]이 없으므로 에러가 떠서 try처리
            if token[a - 1] in prenegative: #'안 좋다'가 원본일 때 '좋다'를 보고있는 차례에서 앞에 '안' 이 있으면 부정으로
                adjust.append(int(i) * -1)
            else:
                adjust.append(i)
        except:
            adjust.append(i)
            
    adjust2 = []
    for a, i in enumerate(adjust):        
        try: 
            if token[a + 1] in sunegative: #'좋지 않다'가 원본일 때 '좋다'를 보고있는 차례에서 뒤에 '않다'가 있으면 부정으로 
                adjust2.append(int(i) * -1)
            else:
                adjust2.append(i)
        except:
            adjust2.append(i)
    
    while True:
        try:
            adjust2.remove("None") #계산값이 None으로 나온 건 제거 #숫자 계산에 글자가 나오면 안됨
        except:
            break
    
    test2 = [] #위에 none code 제거에서 빈리스트 등장하는 경우가 있어서 test2로 다시 append해줌
    for i in adjust2:
        test2.append(int(i)) 
        
    try:
        # 이중문장, 이어진 문장의 경우 '사과는 좋은데 바나나는 싫어' 감성점수 주기 애매
        # 선행연구에서 서로 다른 감정이 동시에 존재할 때 해당 문장을 대표하는 핵심 감정이 무엇이냐를 추출하기 위해서는 
        # 마지막 감성어에 3배 정도의 가중치를 주는 것이 좋다
        # 따라서, 가장 뒤에있는 감정에 가중치 3을 곱함
        test2[-1] = test2[-1] * 3
    except:
        pass

    try:
        return sum(test2)/len(test2) #한 문장안에 여러개의 토큰들이 갖는 감성점수를 토큰의 갯수로 나눠서 평균으로 문장 점수 출력
    except: 
        return 0

#### 2를 가지고 document(문단)의 감성점수 도출

In [11]:
# 원문의 앞뒤 문장을 고려하여 해당 키워드를 포함하는 문장을 감성분석하는 함수
# 문장의 context를 반영한 감성점수를 뽑기 위해서 해당 문장 앞뒤에 있는 context를 구해서 감성점수를 내자

# df는 SNA 단계의 결과로 나온 텍스트 데이터
# keyword는 문자열 단어

def keyword_context_sentiment(df, keyword):
    
    reg_keyword = '\s{}\s|\s{}$|^{}\s'.format(keyword,keyword,keyword)
    
    #reg_keyword 가지고 있으면 target으로
    target = df.document_sentence_for_senti[df.document_sentence_for_senti.str.contains(reg_keyword,regex=True)].to_list() 

    target = list(set(target)) #중복 제거
    result = []
    dexx = []
    
    #if문 이유는 키워드를 포함하는 문장이 어떤 문단에 포함돼있냐에 따라 다른 점수가 나올 수 있음
    if len(target) == 0: # 키워드 포함하는 문장이 없는 경우
        dexx.append(keyword)
        result.append(0)
        
        final_result = {"index" : dexx, "sentiment" : result}
        final_result = pd.DataFrame(final_result)
    
    else: # 키워드 포함하는 문장이 하나일 경우
        for i in target:
            
            gram = df[df["document_sentence_for_senti"] == i] 
            
            
            if len(gram) == 1: #하나만 나올 때는 그냥 계산
                dex = int(gram.index[0]) #인덱스 변수 설정
                
                if dex == df.index[0]: # 맨 앞 인덱스 #맨 앞 문단의 맨 앞 문장 경우 고려 [dex-1]이 정의되지 않아 오류 발생. 차단

                    if df.loc[dex]["document_num"] != df.loc[dex + 1]["document_num"]:
                        consider = df.loc[dex]["document_sentence_for_senti"]
                    else:
                        consider = df.loc[[dex, dex + 1]]["document_sentence_for_senti"]

                elif dex == df.index[-1]: #맨 뒤 인덱스 #맨 뒤 문단의 맨 뒤 문장 고려 [dex+1]이 정의되지 않아 오류 발생. 차단
                    
                    if df.loc[dex - 1]["document_num"] != df.loc[dex]["document_num"]:
                        consider = df.loc[dex]["document_sentence_for_senti"]
                    else:
                        consider = df.loc[[dex - 1, dex]]["document_sentence_for_senti"]
                
                elif df.loc[dex - 1]["document_num"] != df.loc[dex]["document_num"]:
                    #해당 문장이 문단에 맨 앞인지 맨 뒤인지를 고려한 것
                    #해당 문장이 맨 앞 문장이면 인덱스가 앞으로 가면 문단 정보가 바뀜
                    if df.loc[dex]["document_num"] != df.loc[dex + 1]["document_num"]:
                        consider = df.loc[dex]["document_sentence_for_senti"]
                    else:
                        consider = df.loc[[dex, dex + 1]]["document_sentence_for_senti"]
                        
                elif df.loc[dex]["document_num"] != df.loc[dex + 1]["document_num"]:
                    #해당 문장이 맨 뒤 문장이면 인덱스가 뒤로 가면 문단 정보가 바뀜
                    consider = df.loc[[dex - 1, dex]]["document_sentence_for_senti"]
                    
                else: # 위에 경우가 모두 아니면 해당 문장, 앞, 뒤를 모두 고려한 점수
                    consider = df.loc[[dex - 1, dex, dex + 1]]["document_sentence_for_senti"]

                    
                try:
                    consider = pd.DataFrame(consider) #1문단 1문장일 경우 
                except:
                    consider = pd.DataFrame([consider]) #1문단 여러 문장일 경우 (문장이 중간에 있어서 앞뒤 계산이 되는 경우)

                consider.columns = ["document_sentence_for_senti"]

                sentscore = 0
                for i in consider["document_sentence_for_senti"]:
                    sentscore += score_sentence(i)
                result.append(sentscore/len(consider))  # 키워드가 포함된 문장 앞뒤 문장 각각의 문장 감성점수의 평균
                dexx.append(dex)

                
            else: # 키워드를 포함한 문장이 2개 이상일때 #리스트로 묶어서 동시에 고려
                for dex in list(gram.index): 
                    
                    if dex == df.index[0]: # 0번 인덱스
                        if df.loc[dex]["document_num"] != df.loc[dex + 1]["document_num"]:
                            consider = df.loc[dex]["document_sentence_for_senti"]
                        else:
                            consider = df.loc[[dex, dex + 1]]["document_sentence_for_senti"]

                    elif dex == df.index[-1]: #맨뒤 인덱스
                        if df.loc[dex - 1]["document_num"] != df.loc[dex]["document_num"]:
                            consider = df.loc[dex]["document_sentence_for_senti"]
                        else:
                            consider = df.loc[[dex - 1, dex]]["document_sentence_for_senti"]
                    
                    elif df.loc[dex - 1]["document_num"] != df.loc[dex]["document_num"]:
                        if df.loc[dex]["document_num"] != df.loc[dex + 1]["document_num"]:
                            consider = df.loc[dex]["document_sentence_for_senti"]
                        else:
                            consider = df.loc[[dex, dex + 1]]["document_sentence_for_senti"]
                            
                    elif df.loc[dex]["document_num"] != df.loc[dex + 1]["document_num"]:
                        consider = df.loc[[dex - 1, dex]]["document_sentence_for_senti"]
                        
                    else:
                        consider = df.loc[[dex - 1, dex, dex + 1]]["document_sentence_for_senti"]

                    try:
                        consider = pd.DataFrame(consider)
                        
                    except:
                        consider = pd.DataFrame([consider])

                    consider.columns = ["document_sentence_for_senti"]

                    sentscore = 0
                    
                    for i in consider["document_sentence_for_senti"]: 
                        sentscore += score_sentence(i)
                        
                    result.append(sentscore/len(consider))
                    dexx.append(dex)
            
        final_result = {"index" : dexx, "sentiment" : result}
        final_result = pd.DataFrame(final_result)

    return final_result.sentiment.mean()

### 데이터 적용

[cluster0_ms_micro1.csv 설명]

- 명품세탁 키워드로 수집한 ward clustering 1 에 마이크로세그먼트 1 에 할당된 원문임
- documnet_num : 행이 문장 단위로 쪼개져 있어서 문단 번호를 보고 같은 번호끼리 하나의 문단이었음을 확인
- micro_cluster : 마이크로세그먼트 1
- doc_cluster_0 : 명품세탁 ward clustering 1에서 마이크로세그먼트가 4개로 분리됐는데 그 중 0에 할당될 가중치
- doc_cluster_1 : 1에 할당될 가중치(SNA 후 원문 복구할때 도출된 계산), 여기서는 doc_cluster_1이 0,2,3보다 큼
- doc_cluster_2 : 2에 할당될 가중치
- doc_cluster_3 : 3에 할당될 가중치
- document_sentence : 문장 원본

In [5]:
#데이터 불러오기

d01 = pd.read_csv("cluster0_ms_micro1.csv", index_col=0)
#d02 = pd.read_csv("cluster0_ms_micro2.csv")
#d10 = pd.read_csv("cluster1_ms_micro0.csv")
#d11 = pd.read_csv("cluster1_ms_micro1.csv")
#d12 = pd.read_csv("cluster1_ms_micro2.csv")

#dd = [d01,d02,d10,d11,d12]
#tt = [4,2,7,4,3]

dd = [d01] #SNA후 원문 복구된 파일 dataset
tt = [4] #앞에 LDA 코드에서 선정된 토픽갯수가 여기에 들어감 

formemo =['명품세탁 0_1번 4개토픽'] #각 마이크로 세그먼트마다 토픽 갯수 다름(각 데이터마다 토픽 갯수 직접 판단)
          #,'명품세탁 0_2번 2개토픽','명품세탁 1_0번 7개토픽','명품세탁 1_1번 4개토픽','명품세탁 1_2번 3개토픽']

d01.reset_index(inplace=True, drop=True) #인덱스 재설정
d01

,document_num,micro_cluster,doc_cluster_0,doc_cluster_1,doc_cluster_2,doc_cluster_3,document_sentence
0,15,1,14.591668,38.712945,19.029572,26.170874,명품스니커즈 황변제거 명품일수록 좋은재질이다오늘작업한 스니커즈는 가격대가 꽤나갑니...
1,15,1,14.591668,38.712945,19.029572,26.170874,히지만 원단에 필링까지 생기니 말이죠작업을더 해보려 하였으나 워낙고가에신품이라 여...
2,15,1,14.591668,38.712945,19.029572,26.170874,내가 빼줬잖아 돈 받으면 반땅 명품 황변 제거 방법 얘기 해 주시면 복 아낌없이 ...
3,15,1,14.591668,38.712945,19.029572,26.170874,아 계좌번호 올려드려야겟죠 사장님
4,25,1,129.149868,182.991209,10.467515,107.835749,명품패딩 캐나다구스 세탁표시 바로 이해하기 캐나다 구스 품질표시입니다아래 세탁기호...
...,...,...,...,...,...,...,...
3874,2827,1,111.134678,115.567792,33.466962,41.192188,공공장소에서는 아닌것 같네요
3875,2827,1,111.134678,115.567792,33.466962,41.192188,저흰 각자 속옷 가져가서 집에서 세탁 건조해 오는데요이상하신 분이네요
3876,2827,1,111.134678,115.567792,33.466962,41.192188,속옷은 본인이 빨아서 널어야지 참상식 이하이네요몰래 버리세요
3877,2827,1,111.134678,115.567792,33.466962,41.192188,우린 조리사가 위생복하고 속옷을 같이 돌리네요자기가 입고다니는 옷도 여러벌 빨아서 ...


[과정]

1. 여러 dataset 한번에 처리하려고 for문에서 시작
2. 문장 단위로 쪼개져 있던 데이터 문단 단위로 변환
3. LDA에서 만들어지는 행렬 등을 사용하기 위해서 LDA 진행
4. importance 계산
5. sentiment 계산

In [13]:
for order in range(len(dd)): 
    
    memo = formemo[order] #여러개 한번에 처리할때 순서 지정
    df = dd[order]
    TOPIC_NUM = tt[order]
    
    data = []
    
    #문장 단위로 쪼개져 있던 데이터 문단 단위로 변환
    for i in df["document_num"].unique(): # 순수 문단번호
        result = "" 
        for w in df[df["document_num"] == i].values: 
            result += w[-1] # 맨뒤가 document_sentence임
            result += " " # document_sentence가 서로 띄어쓰기 되도록
        data.append(result)
    data = pd.DataFrame(data, columns = ["moondan"])
    data.index = df["document_num"].unique()
    data = data.dropna() #문단단위로 변환된 데이터

    okt_pos = []
    for i in tqdm(range(len(df))):
        okt_pos.append(preprocess_okt_for_sentiment(df['document_sentence'].iloc[i]))
    df['document_sentence_for_senti'] = okt_pos 
    df.document_sentence_for_senti  # LDA의 토픽 키워드를 매핑시켜서 키워드별 감성점수 구할 sentence들

    
    #정해진 topic num 으로 LDA 진행
    #LDA로 도출되는 정보들을 사용해야하기 때문
    tf_vect = TfidfVectorizer(tokenizer=preprocess_okt_for_sentiment, ngram_range=(2,3), min_df=5, max_df=20000) # n_gram 설정
    dtm = tf_vect.fit_transform(data['moondan']) #LDA 

    lda = LatentDirichletAllocation(n_components=TOPIC_NUM, random_state=0, learning_method='batch', max_iter=20)
    lda.fit(dtm)
    # print(dtm)  

    topic_document_matrix = lda.transform(dtm).transpose() # topic_document_matrix

    
    #importance : 토픽별 비중 계산
    
    'importance'
    imp = []
    for i in range(topic_document_matrix.shape[0]): # 토픽마다 돌아가면서
        imp.append(sum(topic_document_matrix[i])) #토픽_도큐먼트 행렬 가로기준으로 합
    imp = imp/sum(imp) # 전체 토픽 분의 각 토픽의 비중
    imp


    # topic_keyword_matrix 각 키워드 평균 감성점수랑 내적할 벡터
    # tf_vect.get_feature_names()  이거랑 각 topic_keyword_matrix의 순서가 같음

    topic_keyword_matrix  = lda.components_ # topic-keyword matrix  행이 토픽, 열이 바이,트라이그램

    keyword_sentiment_vector = []
    for keyword in tqdm(tf_vect.get_feature_names()):
        keyword_sentiment_vector.append(keyword_context_sentiment(df,keyword))

    sat = []
    for i in range(topic_keyword_matrix.shape[0]): # i 는 0~4
        topic_keyword_vector = topic_keyword_matrix[i] # 각 토픽의 키워드 벡터
        sat.append(np.inner(topic_keyword_vector,keyword_sentiment_vector))
    
    imp, sat
    
    f = open("명품세탁중요도만족도.txt", 'a')
    f.write(memo)
    f.write('\n')
    f.write('중요도')
    f.write(str(imp))
    f.write('\n')
    f.write('만족도')
    f.write(str(sat))
    f.write('\n')
    f.close()

100%|██████████████████████████████████████████████████████████████████████████████| 5756/5756 [17:34<00:00,  5.46it/s]


=> txt 파일에 action별 중요도와 만족도 점수가 저장됨. 폴더에서 확인